# INFOMDWR – Assignment 2: Data Integration & Preparation

## Introduction
In this assignment, you will work on different tasks that are related to data preparation including data profiling, finding records that refer to same entity, and computing the correlation between different attributes. The datasets that you need to work on are available online (links are provided).

In [83]:
# Import the necessary libraries

import re
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from typing import Any
from py_stringmatching.similarity_measure.levenshtein import Levenshtein
from py_stringmatching.similarity_measure.jaro import Jaro
from py_stringmatching.similarity_measure.affine import Affine
from itertools import combinations

In [2]:
# Load the csv into a dataframe and print a part of the dataframe

df = pd.read_csv("dft-road-casualty-statistics-collision-2024.csv", low_memory = False)
df

,collision_index,collision_year,collision_ref_no,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,collision_severity,number_of_vehicles,...,carriageway_hazards_historic,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,enhanced_severity_collision,collision_injury_based,collision_adjusted_severity_serious,collision_adjusted_severity_slight
0,202417H103224,2024,17H103224,448894,532505,-1.24312,54.68523,17,3,2,...,-1,0,1,2,2,E01011983,3,1,0.000000,1.000000
1,202417M217924,2024,17M217924,452135,519436,-1.19517,54.56747,17,2,2,...,-1,0,1,3,2,E01012061,7,1,1.000000,0.000000
2,202417S204524,2024,17S204524,445427,522924,-1.29837,54.59946,17,3,2,...,0,0,2,1,2,E01012280,-1,0,0.111621,0.888379
3,2024481510889,2024,481510889,533587,181174,-0.07626,51.51371,48,2,1,...,-1,0,1,1,2,E01000005,7,1,1.000000,0.000000
4,2024481563500,2024,481563500,532676,180902,-0.08948,51.51148,48,2,1,...,-1,0,1,1,2,E01032739,5,1,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100922,2024991432330,2024,991432330,255558,666585,-4.31001,55.87072,99,2,1,...,-1,0,1,1,-1,-1,6,1,1.000000,0.000000
100923,2024991466055,2024,991466055,258265,666632,-4.26681,55.87194,99,3,1,...,-1,0,1,1,-1,-1,3,1,0.000000,1.000000
100924,2024991485880,2024,991485880,264189,664898,-4.17134,55.85808,99,2,2,...,-1,0,1,1,-1,-1,7,1,1.000000,0.000000
100925,2024991436758,2024,991436758,291004,658589,-3.74065,55.80823,99,3,2,...,-1,0,2,1,-1,-1,3,1,0.000000,1.000000


## Task 1: Profiling relational data

For this task, download and read the paper about profiling relational data, select a set of summary statistics about the data (minimum of 10 different values) and write Python code to compute these quantities for a dataset of your choice. Preferably, you can use one of the csv files from the road safety dataset. Explain the importance of each summary statistic that you selected in understanding the characteristics of the dataset.

*Note: Computing the same statistical quantity on multiple columns of the dataset will be counted only once.*

In [3]:
# Set up a dictionary to hold the summary statistics

summary_statistics: dict[str, Any] = {
    "mean": None,
    "median": None,
    "mode": None,
    "minimum": None,
    "maximum": None,
    "range": None,
    "variance": None,
    "standard_deviation": None,
    "records": None,
    "number_of_missing_records": None,
}

In [4]:
# Fill the dictionary for summary statistics

summary_statistics["mean"] = df["speed_limit"].mean()                                           # Mean of the speed limit
summary_statistics["median"] = df["speed_limit"].median()                                       # Median of the speed limit 
summary_statistics["mode"] = df["speed_limit"].mode()                                           # Mode of the speed limit
summary_statistics["minimum"] = df["speed_limit"].min()                                         # Min of the speed limit
summary_statistics["maximum"] = df["speed_limit"].max()                                         # Max of the speed limit
summary_statistics["range"] = abs(df["speed_limit"].max()) + abs(df["speed_limit"].min())       # Range of the speed limit, thus abs(max)+ and(min))
summary_statistics["variance"] = df["speed_limit"].var()                                        # Variance of the speed limit
summary_statistics["standard_deviation"] = df["speed_limit"].std()                              # Standard deviation of the speed limit
summary_statistics["records"] = len(df["speed_limit"])                                          # Number of rows in the dataframe
summary_statistics["number_of_missing_records"] = df.isnull().sum().sum()                       # Number of missing values in the entire dataframe

summary_statistics

{'mean': 35.87966550080751,
 'median': 30.0,
 'mode': 0    30
 Name: speed_limit, dtype: int64,
 'minimum': -1,
 'maximum': 70,
 'range': 71,
 'variance': 211.30421831349014,
 'standard_deviation': 14.536306900774012,
 'records': 100927,
 'number_of_missing_records': 3}

## Task 2: Entity resolution

### Part 1:

Write a Python code to compare every single record in the dataset (ACM.csv) with all the records in (DBLP2.csv) and find the similar records (records that represent the same publication). To compare two records, follow the steps:

In [5]:
# Import the datasets

df_acm = pd.read_csv("ACM.csv")
df_dblp2 = pd.read_csv("DBLP2.csv", encoding="latin1")


In [6]:
# Show the dataframe of ACM

df_acm

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
4,304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999
...,...,...,...,...,...
2289,672977,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",Very Large Data Bases,1994
2290,950482,Guest editorial,"Philip A. Bernstein, Yannis Ioannidis, Raghu R...",The VLDB Journal &mdash; The International Jou...,2003
2291,672980,GraphDB: Modeling and Querying Graphs in Datab...,Ralf Hartmut G&#252;ting,Very Large Data Bases,1994
2292,945741,Review of The data warehouse toolkit: the comp...,Alexander A. Anisimov,ACM SIGMOD Record,2003


In [7]:
# Show the dataframe of dblp2

df_dblp2

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002
3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G...",VLDB,1996
4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995
...,...,...,...,...,...
2611,journals/tods/KarpSP03,A simple algorithm for finding frequent elemen...,"Scott Shenker, Christos H. Papadimitriou, Rich...",ACM Trans. Database Syst.,2003
2612,conf/vldb/LimWV03,SASH: A Self-Adaptive Histogram Set for Dynami...,"Lipyeow Lim, Min Wang, Jeffrey Scott Vitter",VLDB,2003
2613,journals/tods/ChakrabartiKMP02,Locally adaptive dimensionality reduction for ...,"Kaushik Chakrabarti, Eamonn J. Keogh, Michael ...",ACM Trans. Database Syst.,2002
2614,journals/sigmod/Snodgrass01,Chair's Message,Richard T. Snodgrass,SIGMOD Record,2001


## Part 1

### A. 

Ignore the pub_id

In [8]:
df_acm.drop("id", axis=1, inplace = True)

In [9]:
df_dblp2.drop("id", axis=1, inplace = True)

### B. 

Change all alphabetical characters into lowercase.

In [10]:
for col in df_acm.columns:
    df_acm[col] = df_acm[col].astype(str).str.lower()
for col in df_dblp2.columns:
    df_dblp2[col] = df_dblp2[col].astype(str).str.lower()

In [11]:
df_acm

,title,authors,venue,year
0,the wasa2 object-oriented workflow management ...,"gottfried vossen, mathias weske",international conference on management of data,1999
1,a user-centered interface for querying distrib...,"isabel f. cruz, kimberly m. james",international conference on management of data,1999
2,"world wide database-integrating the web, corba...","athman bouguettaya, boualem benatallah, lily h...",international conference on management of data,1999
3,xml-based information mediation with mix,"chaitan baru, amarnath gupta, bertram lud&#228...",international conference on management of data,1999
4,the ccube constraint object-oriented database ...,"alexander brodsky, victor e. segal, jia chen, ...",international conference on management of data,1999
...,...,...,...,...
2289,dual-buffering strategies in object bases,"alfons kemper, donald kossmann",very large data bases,1994
2290,guest editorial,"philip a. bernstein, yannis ioannidis, raghu r...",the vldb journal &mdash; the international jou...,2003
2291,graphdb: modeling and querying graphs in datab...,ralf hartmut g&#252;ting,very large data bases,1994
2292,review of the data warehouse toolkit: the comp...,alexander a. anisimov,acm sigmod record,2003


In [12]:
df_dblp2

,title,authors,venue,year
0,semantic integration of environmental models f...,d. scott mackay,sigmod record,1999
1,estimation of query-result distribution and it...,"viswanath poosala, yannis e. ioannidis",vldb,1996
2,incremental maintenance for non-distributive a...,"themistoklis palpanas, richard sidle, hamid pi...",vldb,2002
3,cost-based selection of path expression proces...,"zhao-hui tang, georges gardarin, jean-robert g...",vldb,1996
4,benchmarking spatial join operations with spat...,"erik g. hoel, hanan samet",vldb,1995
...,...,...,...,...
2611,a simple algorithm for finding frequent elemen...,"scott shenker, christos h. papadimitriou, rich...",acm trans. database syst.,2003
2612,sash: a self-adaptive histogram set for dynami...,"lipyeow lim, min wang, jeffrey scott vitter",vldb,2003
2613,locally adaptive dimensionality reduction for ...,"kaushik chakrabarti, eamonn j. keogh, michael ...",acm trans. database syst.,2002
2614,chair's message,richard t. snodgrass,sigmod record,2001


### C. 

Convert multiple spaces to one

In [13]:
for col in df_acm.columns:
    df_acm[col].replace(r'\s {2,}', ' ', regex=True)

df_acm

,title,authors,venue,year
0,the wasa2 object-oriented workflow management ...,"gottfried vossen, mathias weske",international conference on management of data,1999
1,a user-centered interface for querying distrib...,"isabel f. cruz, kimberly m. james",international conference on management of data,1999
2,"world wide database-integrating the web, corba...","athman bouguettaya, boualem benatallah, lily h...",international conference on management of data,1999
3,xml-based information mediation with mix,"chaitan baru, amarnath gupta, bertram lud&#228...",international conference on management of data,1999
4,the ccube constraint object-oriented database ...,"alexander brodsky, victor e. segal, jia chen, ...",international conference on management of data,1999
...,...,...,...,...
2289,dual-buffering strategies in object bases,"alfons kemper, donald kossmann",very large data bases,1994
2290,guest editorial,"philip a. bernstein, yannis ioannidis, raghu r...",the vldb journal &mdash; the international jou...,2003
2291,graphdb: modeling and querying graphs in datab...,ralf hartmut g&#252;ting,very large data bases,1994
2292,review of the data warehouse toolkit: the comp...,alexander a. anisimov,acm sigmod record,2003


In [14]:
for col in df_dblp2.columns:
    df_dblp2[col].replace(r'\s {2,}', ' ', regex=True)

df_dblp2

,title,authors,venue,year
0,semantic integration of environmental models f...,d. scott mackay,sigmod record,1999
1,estimation of query-result distribution and it...,"viswanath poosala, yannis e. ioannidis",vldb,1996
2,incremental maintenance for non-distributive a...,"themistoklis palpanas, richard sidle, hamid pi...",vldb,2002
3,cost-based selection of path expression proces...,"zhao-hui tang, georges gardarin, jean-robert g...",vldb,1996
4,benchmarking spatial join operations with spat...,"erik g. hoel, hanan samet",vldb,1995
...,...,...,...,...
2611,a simple algorithm for finding frequent elemen...,"scott shenker, christos h. papadimitriou, rich...",acm trans. database syst.,2003
2612,sash: a self-adaptive histogram set for dynami...,"lipyeow lim, min wang, jeffrey scott vitter",vldb,2003
2613,locally adaptive dimensionality reduction for ...,"kaushik chakrabarti, eamonn j. keogh, michael ...",acm trans. database syst.,2002
2614,chair's message,richard t. snodgrass,sigmod record,2001


### D.

Use Levenshtein similarity $(L_{sim}(S_1, S_2) = 1 - \frac{MED(S_1, S_2}{MAX(|S_1|, |S_2|})$
for comparing the values in the **title** attribute and compute the score $(s_t)$. (MED refers to the minimum edit distance and $|S_i|$ is the number of characters in string $S_i$).


In [ ]:
# Levenshtein similarity for title comparison
def levenshtein_similarity(s1, s2):
    lev = Levenshtein()
    distance = lev.get_raw_score(s1, s2)
    max_len = max(len(s1), len(s2))
    if max_len == 0:
        return 1.0
    return 1 - (distance / max_len)

# Compare titles from ACM and DBLP datasets
title_similarity = {}
for i, title1 in enumerate(df_acm["title"]):
    for j, title2 in enumerate(df_dblp2["title"]):
        similarity = levenshtein_similarity(title1, title2)
        title_similarity[(i, j)] = similarity

# Find the pair with highest similarity
max_title_sim = max(title_similarity, key=title_similarity.get)
print(f"Highest title similarity: {title_similarity[max_title_sim]}")
print(f"ACM title: {df_acm.iloc[max_title_sim[0]]['title']}")
print(f"DBLP title: {df_dblp2.iloc[max_title_sim[1]]['title']}")


Highest title similarity: 1.0000
ACM title: the wasa2 object-oriented workflow management system
DBLP title: the wasa2 object-oriented workflow management system


### E. 

Use Jaro similarity to compare the values in the **authors** field and compute $(S_a)$.

In [ ]:
# Jaro similarity for authors comparison
jaro = Jaro()

# Compare authors from ACM and DBLP datasets
authors_similarity = {}
for i, author1 in enumerate(df_acm["authors"]):
    for j, author2 in enumerate(df_dblp2["authors"]):
        similarity = jaro.get_sim_score(author1, author2)
        authors_similarity[(i, j)] = similarity

# Find the pair with highest similarity
max_authors_sim = max(authors_similarity, key=authors_similarity.get)
print(f"Highest authors similarity: {authors_similarity[max_authors_sim]}")
print(f"ACM author: {df_acm.iloc[max_authors_sim[0]]['authors']}")
print(f"DBLP author: {df_dblp2.iloc[max_authors_sim[1]]['authors']}")


Highest authors similarity: 1.0000
ACM author: martin bichler, arie segev, j. leon zhao
DBLP author: martin bichler, arie segev, j. leon zhao


### F.

Use a modified version of the affine similarity that is scaled to the interval [0, 1] for the venue attribute $(S_c)$.

In [24]:
affine = Affine()

venue_similarity = {}
for i, venue1 in enumerate(df_acm["venue"]):
    for j, venue2 in enumerate(df_dblp2["venue"]):
        similarity = affine.get_raw_score(str(venue1), str(venue2))
        venue_similarity[(i, j)] = similarity

max_venue_sim = max(venue_similarity, key=venue_similarity.get)
print(f"Highest venue similarity: {venue_similarity[max_venue_sim]:.4f}")
print(f"ACM venue: {df_acm.iloc[max_venue_sim[0]]['venue']}")
print(f"DBLP venue: {df_dblp2.iloc[max_venue_sim[1]]['venue']}")


Highest venue similarity: 12.5000
ACM venue: acm transactions on database systems (tods) 
DBLP venue: acm trans. database syst.


### G.

Use Match (1) / Mismatch (0) for the year $(S_y)$.

In [ ]:
# Match/Mismatch for year comparison
def year_similarity(year1, year2):
    """Return 1 if years match exactly, 0 otherwise"""
    return 1.0 if year1 == year2 else 0.0

# Compare years from ACM and DBLP datasets
year_similarity_dict = {}
for i, year1 in enumerate(df_acm["year"]):
    for j, year2 in enumerate(df_dblp2["year"]):
        similarity = year_similarity(year1, year2)
        year_similarity_dict[(i, j)] = similarity

# Count exact matches
exact_matches = sum(1 for sim in year_similarity_dict.values() if sim == 1.0)
total_comparisons = len(year_similarity_dict)
print(f"Exact year matches: {exact_matches} out of {total_comparisons} comparisons")
print(f"Match rate: {exact_matches/total_comparisons}")


Exact year matches: 601284 out of 6001104 comparisons
Match rate: 0.1002


### H.

Use the formula $\text{rec\_sim} = w_1 * s_1 + w_2 * s_2 + w_3 * s_3 + w_4 * s_4$ to combine the scores and compute the final score, where $\sum^4_{i=1}w_i=1$.

In [28]:
# Combine scores using weighted formula: rec_sim = w1*s1 + w2*s2 + w3*s3 + w4*s4
def combine_scores(s_t, s_a, s_c, s_y, w1=0.3, w2=0.3, w3=0.2, w4=0.2):
    """Combine similarity scores with weights where w1+w2+w3+w4=1"""
    return w1 * s_t + w2 * s_a + w3 * s_c + w4 * s_y

# Calculate combined similarity for all pairs
combined_similarity = {}
for i in range(len(df_acm)):
    for j in range(len(df_dblp2)):
        s_t = title_similarity.get((i, j), 0)
        s_a = authors_similarity.get((i, j), 0)
        s_c = venue_similarity.get((i, j), 0)
        s_y = year_similarity_dict.get((i, j), 0)
        
        # Combine scores
        combined_score = combine_scores(s_t, s_a, s_c, s_y)
        combined_similarity[(i, j)] = combined_score

# Find the pair with highest combined similarity
max_combined = max(combined_similarity, key=combined_similarity.get)
print(f"Highest combined similarity: {combined_similarity[max_combined]}")
print(f"ACM record: {df_acm.iloc[max_combined[0]]['title']}")
print(f"DBLP record: {df_dblp2.iloc[max_combined[1]]['title']}")


Highest combined similarity: 3.3000000000000003
ACM record: security of random data perturbation methods
DBLP record: security of random data perturbation methods


### I.

Report the records with rec_sim > 0.7 as duplicate records by storing the ids of both records in a list

In [43]:
# Report records with rec_sim > 0.7 as duplicate records
def report_duplicates(threshold=0.7):
    duplicates = []
    for (i, j), score in combined_similarity.items():
        if score > threshold:
            duplicates.append((i, j, score))
    
    # Sort by similarity score (descending)
    duplicates.sort(key=lambda x: x[2], reverse=True)
    
    print(f"Found {len(duplicates)} duplicate pairs with similarity > {threshold}")

    return duplicates

# Report duplicates
duplicate_pairs = report_duplicates()


Found 835714 duplicate pairs with similarity > 0.7


### J.

In the table `DBLP-ACM_perfectMapping.csv`, you can find the actual mappings (the ids of the correct duplicate records). Compute the precision of this method by counting the number of duplicate records that you discovered correctly. That is, among all the reported similar records by your method, how many pairs exist in the file `DBLP-ACM_perfectMapping.csv`

In [86]:
# Reload the original datasets to get the correct IDs
df_acm_original = pd.read_csv("ACM.csv")
df_dblp2_original = pd.read_csv("DBLP2.csv", encoding="latin1")

# Load the perfect mapping file
perfect_mapping = pd.read_csv("DBLP-ACM_perfectMapping.csv")

# Convert perfect mapping to set of tuples for faster lookup
perfect_pairs = set()
for _, row in perfect_mapping.iterrows():
    perfect_pairs.add((row['idACM'], row['idDBLP']))

# Calculate precision
def calculate_precision(reported_duplicates, perfect_pairs, df_acm_original, df_dblp2_original):
    """Calculate precision: TP / (TP + FP)"""
    true_positives = 0
    false_positives = 0
    
    for acm_idx, dblp_idx, score in reported_duplicates:
        # Get the actual IDs from the original datasets
        acm_id = df_acm_original.iloc[acm_idx]['id']
        dblp_id = df_dblp2_original.iloc[dblp_idx]['id']
        
        if (acm_id, dblp_id) in perfect_pairs:
            true_positives += 1
        else:
            false_positives += 1
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    return precision, true_positives, false_positives

# Calculate precision
precision, tp, fp = calculate_precision(duplicate_pairs, perfect_pairs, df_acm_original, df_dblp2_original)
print(f"Precision: {precision}")
print(f"True Positives: {tp}")
print(f"False Positives: {fp}")
print(f"Total reported duplicates: {len(duplicate_pairs)}")

Precision: 0.0007
True Positives: 592
False Positives: 835122
Total reported duplicates: 835714


### K.

Record the running time of the method. You can observe that the program takes a long time to get the results. What can you do to reduce the running time? (Just provide clear discussion – no need for implementing the ideas.)

Precision: 0.0007
True Positives: 592
False Positives: 835122
Total reported duplicates: 835714


## Part 2

### 1.

Concatenate the values in each record into one single string.

In [73]:
def concatenate_record(record):
    """Concatenate all field values of a record into a single string"""
    return ' '.join(str(value) for value in record.values)

acm_concatenated = []
for idx, record in df_acm.iterrows():
    acm_concatenated.append(concatenate_record(record))
  
dblp_concatenated = []
for idx, record in df_dblp2.iterrows():
    dblp_concatenated.append(concatenate_record(record))

print(acm_concatenated)
print(dblp_concatenated)

['the wasa2 object-oriented workflow management system gottfried vossen, mathias weske international conference on management of data 1999 0', 'a user-centered interface for querying distributed multimedia databases isabel f. cruz, kimberly m. james international conference on management of data 1999 1', 'world wide database-integrating the web, corba and databases athman bouguettaya, boualem benatallah, lily hendra, james beard, kevin smith, mourad quzzani international conference on management of data 1999 2', 'xml-based information mediation with mix chaitan baru, amarnath gupta, bertram lud&#228;scher, richard marciano, yannis papakonstantinou, pavel velikhov, vincent chu international conference on management of data 1999 3', 'the ccube constraint object-oriented database system alexander brodsky, victor e. segal, jia chen, paval a. exarkhopoulo international conference on management of data 1999 4', 'the cornell jaguar project: adding mobility to predator phillippe bonnet, kyle b

### 2.

Change all alphabetical characters into lowercase.

In [72]:
acm_concatenated_lower = [text.lower() for text in acm_concatenated]
dblp_concatenated_lower = [text.lower() for text in dblp_concatenated]


print(acm_concatenated_lower)
print(dblp_concatenated_lower)

['the wasa2 object-oriented workflow management system gottfried vossen, mathias weske international conference on management of data 1999 0', 'a user-centered interface for querying distributed multimedia databases isabel f. cruz, kimberly m. james international conference on management of data 1999 1', 'world wide database-integrating the web, corba and databases athman bouguettaya, boualem benatallah, lily hendra, james beard, kevin smith, mourad quzzani international conference on management of data 1999 2', 'xml-based information mediation with mix chaitan baru, amarnath gupta, bertram lud&#228;scher, richard marciano, yannis papakonstantinou, pavel velikhov, vincent chu international conference on management of data 1999 3', 'the ccube constraint object-oriented database system alexander brodsky, victor e. segal, jia chen, paval a. exarkhopoulo international conference on management of data 1999 4', 'the cornell jaguar project: adding mobility to predator phillippe bonnet, kyle b

### 3.

Convert multiple spaces to one.

In [71]:
acm_concatenated_clean = [re.sub(r'\s+', ' ', text) for text in acm_concatenated_lower]
dblp_concatenated_clean = [re.sub(r'\s+', ' ', text) for text in dblp_concatenated_lower]

print(acm_concatenated_clean)
print(dblp_concatenated_clean)


['the wasa2 object-oriented workflow management system gottfried vossen, mathias weske international conference on management of data 1999 0', 'a user-centered interface for querying distributed multimedia databases isabel f. cruz, kimberly m. james international conference on management of data 1999 1', 'world wide database-integrating the web, corba and databases athman bouguettaya, boualem benatallah, lily hendra, james beard, kevin smith, mourad quzzani international conference on management of data 1999 2', 'xml-based information mediation with mix chaitan baru, amarnath gupta, bertram lud&#228;scher, richard marciano, yannis papakonstantinou, pavel velikhov, vincent chu international conference on management of data 1999 3', 'the ccube constraint object-oriented database system alexander brodsky, victor e. segal, jia chen, paval a. exarkhopoulo international conference on management of data 1999 4', 'the cornell jaguar project: adding mobility to predator phillippe bonnet, kyle b

### 4.

Combine the records from both tables into one big list as we did during the lab.

In [70]:
# 4. Combine the records from both tables into one big list
all_records = acm_concatenated_clean + dblp_concatenated_clean
all_record_ids = []

# Create IDs for tracking (ACM records get negative IDs, DBLP get positive)
for i, record in enumerate(acm_concatenated_clean):
    all_record_ids.append(f"ACM_{i}")
    
for i, record in enumerate(dblp_concatenated_clean):
    all_record_ids.append(f"DBLP_{i}")


print(all_records)
print(all_record_ids)

['the wasa2 object-oriented workflow management system gottfried vossen, mathias weske international conference on management of data 1999 0', 'a user-centered interface for querying distributed multimedia databases isabel f. cruz, kimberly m. james international conference on management of data 1999 1', 'world wide database-integrating the web, corba and databases athman bouguettaya, boualem benatallah, lily hendra, james beard, kevin smith, mourad quzzani international conference on management of data 1999 2', 'xml-based information mediation with mix chaitan baru, amarnath gupta, bertram lud&#228;scher, richard marciano, yannis papakonstantinou, pavel velikhov, vincent chu international conference on management of data 1999 3', 'the ccube constraint object-oriented database system alexander brodsky, victor e. segal, jia chen, paval a. exarkhopoulo international conference on management of data 1999 4', 'the cornell jaguar project: adding mobility to predator phillippe bonnet, kyle b

### 5.

Use the functions in the tutorials from lab 5 to compute the shingles, the minhash signature and the similarity.

In [77]:
def get_minhash_arr(num_hashes:int,vocab:dict):
    """
    Generates a MinHash array for the given vocabulary.

    This function creates an array where each row represents a hash function and
    each column corresponds to a word in the vocabulary. The values are permutations
    of integers representing the hashed value of each word for that particular hash function.

    Parameters:
    - num_hashes (int): The number of hash functions (rows) to generate for the MinHash array.
    - vocab (dict): The vocabulary where keys are words and values can be any data
      (only keys are used in this function).

    Returns:
    - np.ndarray: The generated MinHash array with `num_hashes` rows and columns equal
      to the size of the vocabulary. Each cell contains the hashed value of the corresponding
      word for the respective hash function.

    Example:
    vocab = {'apple': 1, 'banana': 2}
    get_minhash_arr(2, vocab)
    # Possible output:
    # array([[1, 2],
    #        [2, 1]])
    """
    length = len(vocab.keys())
    arr = np.zeros((num_hashes,length))
    for i in range(num_hashes):
        permutation = np.random.permutation(len(vocab.keys())) + 1
        arr[i,:] = permutation.copy()
    return arr.astype(int)

def get_signature(minhash:np.ndarray, vector:np.ndarray):
    """
    Computes the signature of a given vector using the provided MinHash matrix.

    The function finds the nonzero indices of the vector, extracts the corresponding
    columns from the MinHash matrix, and computes the signature as the minimum value
    across those columns for each row of the MinHash matrix.

    Parameters:
    - minhash (np.ndarray): The MinHash matrix where each column represents a shingle
      and each row represents a hash function.
    - vector (np.ndarray): A vector representing the presence (non-zero values) or
      absence (zero values) of shingles.

    Returns:
    - np.ndarray: The signature vector derived from the MinHash matrix for the provided vector.

    Example:
    minhash = np.array([[2, 3, 4], [5, 6, 7], [8, 9, 10]])
    vector = np.array([0, 1, 0])
    get_signature(minhash, vector)
    output:array([3, 6, 9])
    """
    idx = np.nonzero(vector)[0].tolist()
    shingles = minhash[:,idx]
    signature = np.min(shingles,axis=1)
    return signature

def jaccard_similarity(set1, set2):
    intersection_size = len(set1.intersection(set2))
    union_size = len(set1.union(set2))
    return intersection_size / union_size if union_size != 0 else 0.0

def compute_signature_similarity(signature_1, signature_2):
    """
    Calculate the similarity between two signature matrices using MinHash.

    Parameters:
    - signature_1: First signature matrix as a numpy array.
    - signature_matrix2: Second signature matrix as a numpy array.

    Returns:
    - Estimated Jaccard similarity.
    """
    # Ensure the matrices have the same shape
    if signature_1.shape != signature_2.shape:
        raise ValueError("Both signature matrices must have the same shape.")
    # Count the number of rows where the two matrices agree
    agreement_count = np.sum(signature_1 == signature_2)
    # Calculate the similarity
    similarity = agreement_count / signature_2.shape[0]

    return similarity

def shingle(text: str, k: int)->set:
    """
    Create a set of 'shingles' from the input text using k-shingling.

    Parameters:
        text (str): The input text to be converted into shingles.
        k (int): The length of the shingles (substring size).

    Returns:
        set: A set containing the shingles extracted from the input text.
    """
    shingle_set = []
    for i in range(len(text) - k+1):
        shingle_set.append(text[i:i+k])
    return set(shingle_set)

def build_vocab(shingle_sets: list)->dict:
    """
    Constructs a vocabulary dictionary from a list of shingle sets.

    This function takes a list of shingle sets and creates a unified vocabulary
    dictionary. Each unique shingle across all sets is assigned a unique integer
    identifier.

    Parameters:
    - shingle_sets (list of set): A list containing sets of shingles.

    Returns:
    - dict: A vocabulary dictionary where keys are the unique shingles and values
      are their corresponding unique integer identifiers.

    Example:
    sets = [{"apple", "banana"}, {"banana", "cherry"}]
    build_vocab(sets)
    {'apple': 0, 'cherry': 1, 'banana': 2}  # The exact order might vary due to set behavior
    """
    full_set = {item for set_ in shingle_sets for item in set_}
    vocab = {}
    for i, shingle in enumerate(list(full_set)):
        vocab[shingle] = i
    return vocab

def one_hot(shingles: set, vocab: dict):
    vec = np.zeros(len(vocab))
    for shingle in shingles:
        idx = vocab[shingle]
        vec[idx] = 1
    return vec

# Create shingles for all records
print("Creating shingles...")
all_shingle_sets = []
for i, record in enumerate(all_records):
    shingles = shingle(record, k=3)
    all_shingle_sets.append(shingles)
    
    if i % 500 == 0:
        print(f"Processed {i} records...")

print(f"Created shingles for {len(all_shingle_sets)} records")

# Build vocabulary from all shingle sets
print("Building vocabulary...")
vocab = build_vocab(all_shingle_sets)
print(f"Vocabulary size: {len(vocab)}")

# Generate MinHash array
print("Generating MinHash array...")
num_hashes = 100
minhash_array = get_minhash_arr(num_hashes, vocab)

# Create signatures for all records
print("Creating signatures...")
all_signatures = []
for i, shingle_set in enumerate(all_shingle_sets):
    # Convert shingles to one-hot vector
    vector = one_hot(shingle_set, vocab)
    
    # Get signature using MinHash
    signature = get_signature(minhash_array, vector)
    all_signatures.append(signature)
    
    if i % 500 == 0:
        print(f"Processed {i} signatures...")

print(f"Created signatures for {len(all_signatures)} records")

Creating shingles...
Processed 0 records...
Processed 500 records...
Processed 1000 records...
Processed 1500 records...
Processed 2000 records...
Processed 2500 records...
Processed 3000 records...
Processed 3500 records...
Processed 4000 records...
Processed 4500 records...
Created shingles for 4910 records
Building vocabulary...
Vocabulary size: 10353
Generating MinHash array...
Creating signatures...
Processed 0 signatures...
Processed 500 signatures...
Processed 1000 signatures...
Processed 1500 signatures...
Processed 2000 signatures...
Processed 2500 signatures...
Processed 3000 signatures...
Processed 3500 signatures...
Processed 4000 signatures...
Processed 4500 signatures...
Created signatures for 4910 records


### 6.

Extract the top 2224 candidates from the LSH algorithm, compare them to the actual mappings in the file `DBLP-ACM_perfectMapping.csv` and compute the precision of the method.

In [80]:
class LSH:
    """
    Implements the Locality Sensitive Hashing (LSH) technique for approximate
    nearest neighbor search.
    """
    buckets = []
    counter = 0

    def __init__(self, b: int):
        """
        Initializes the LSH instance with a specified number of bands.

        Parameters:
        - b (int): The number of bands to divide the signature into.
        """
        self.b = b
        for i in range(b):
            self.buckets.append({})

    def make_subvecs(self, signature: np.ndarray) -> np.ndarray:
        """
        Divides a given signature into subvectors based on the number of bands.

        Parameters:
        - signature (np.ndarray): The MinHash signature to be divided.

        Returns:
        - np.ndarray: A stacked array where each row is a subvector of the signature.
        """
        l = len(signature)
        assert l % self.b == 0
        r = int(l / self.b)
        subvecs = []
        for i in range(0, l, r):
            subvecs.append(signature[i:i+r])
        return np.stack(subvecs)

    def add_hash(self, signature: np.ndarray):
        """
        Adds a signature to the appropriate LSH buckets based on its subvectors.

        Parameters:
        - signature (np.ndarray): The MinHash signature to be hashed and added.
        """
        subvecs = self.make_subvecs(signature).astype(str)
        for i, subvec in enumerate(subvecs):
            subvec = ','.join(subvec)
            if subvec not in self.buckets[i].keys():
                self.buckets[i][subvec] = []
            self.buckets[i][subvec].append(self.counter)
        self.counter += 1

    def check_candidates(self) -> set:
        """
        Identifies candidate pairs from the LSH buckets that could be potential near duplicates.

        Returns:
        - set: A set of tuple pairs representing the indices of candidate signatures.
        """
        candidates = []
        for bucket_band in self.buckets:
            keys = bucket_band.keys()
            for bucket in keys:
                hits = bucket_band[bucket]
                if len(hits) > 1:
                    candidates.extend(combinations(hits, 2))
        return set(candidates)

In [82]:
lsh = LSH(20)

for i, signature in enumerate(all_signatures):
    lsh.add_hash(signature)

candidate_pairs = lsh.check_candidates()

candidate_similarities = []
for i, j in candidate_pairs:
    if (i < len(acm_concatenated_clean) and j >= len(acm_concatenated_clean)) or \
       (j < len(acm_concatenated_clean) and i >= len(acm_concatenated_clean)):
        similarity = compute_signature_similarity(all_signatures[i], all_signatures[j])
        candidate_similarities.append((i, j, similarity))

candidate_similarities.sort(key=lambda x: x[2], reverse=True)
top_candidates = candidate_similarities[:2224]

perfect_mapping = pd.read_csv("DBLP-ACM_perfectMapping.csv")

perfect_pairs = set()
for _, row in perfect_mapping.iterrows():
    perfect_pairs.add((row['idACM'], row['idDBLP']))


def calculate_lsh_precision(candidates, perfect_pairs, df_acm_original, df_dblp2_original):
    true_positives = 0
    false_positives = 0
    
    for i, j, _ in candidates:
        if i < len(acm_concatenated_clean):
            acm_idx = i
            dblp_idx = j - len(acm_concatenated_clean)
        else:
            acm_idx = j
            dblp_idx = i - len(acm_concatenated_clean)
        
        acm_id = df_acm_original.iloc[acm_idx]['id']
        dblp_id = df_dblp2_original.iloc[dblp_idx]['id']
        
        if (acm_id, dblp_id) in perfect_pairs:
            true_positives += 1
        else:
            false_positives += 1
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    return precision, true_positives, false_positives

lsh_precision, lsh_tp, lsh_fp = calculate_lsh_precision(top_candidates, perfect_pairs, df_acm_original, df_dblp2_original)
print(f"LSH Precision: {lsh_precision}")
print(f"True Positives: {lsh_tp}")
print(f"False Positives: {lsh_fp}")
print(f"Total candidates: {len(top_candidates)}")

print("Top 10 LSH candidates:")
for i, (idx1, idx2, similarity) in enumerate(top_candidates[:10]):
    if idx1 < len(acm_concatenated_clean):
        acm_idx = idx1
        dblp_idx = idx2 - len(acm_concatenated_clean)
    else:
        acm_idx = idx2
        dblp_idx = idx1 - len(acm_concatenated_clean)
    
    print(f"{i+1}. Similarity: {similarity}")
    print(f"   ACM: {df_acm_original.iloc[acm_idx]['title']}")
    print(f"   DBLP: {df_dblp2_original.iloc[dblp_idx]['title']}")

LSH Precision: 0.970773381294964
True Positives: 2159
False Positives: 65
Total candidates: 2224
Top 10 LSH candidates:
1. Similarity: 0.98
   ACM: Review of The data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) by Ralph Kimball, Margy Ross. John Wiley & Sons, Inc. 2002.
   DBLP: Review of The data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) by Ralph Kimball, Margy Ross. John Wiley & Sons, Inc. 2002
2. Similarity: 0.98
   ACM: Review of The data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) by Ralph Kimball, Margy Ross. John Wiley & Sons, Inc. 2002.
   DBLP: Review of The data warehouse toolkit: the complete guide to dimensional modeling (2nd edition) by Ralph Kimball, Margy Ross. John Wiley & Sons, Inc. 2002
3. Similarity: 0.98
   ACM: Selective information dissemination in P2P networks: problems and solutions
   DBLP: Selective information dissemination in P2P networks: problems and solutions

### 7.

Record the running time of the method.

In [85]:
lsh_start_time = time.time()

all_shingle_sets = []
for i, record in enumerate(all_records):
    shingles = shingle(record, k=3)
    all_shingle_sets.append(shingles)

vocab = build_vocab(all_shingle_sets)

num_hashes = 100
minhash_array = get_minhash_arr(num_hashes, vocab)

all_signatures = []
for i, shingle_set in enumerate(all_shingle_sets):
    vector = one_hot(shingle_set, vocab)
    signature = get_signature(minhash_array, vector)
    all_signatures.append(signature)

num_bands = 20
lsh = LSH(num_bands)

for i, signature in enumerate(all_signatures):
    lsh.add_hash(signature)

candidate_pairs = lsh.check_candidates()

candidate_similarities = []
for i, j in candidate_pairs:
    if (i < len(acm_concatenated_clean) and j >= len(acm_concatenated_clean)) or \
       (j < len(acm_concatenated_clean) and i >= len(acm_concatenated_clean)):
        similarity = compute_signature_similarity(all_signatures[i], all_signatures[j])
        candidate_similarities.append((i, j, similarity))

candidate_similarities.sort(key=lambda x: x[2], reverse=True)
top_candidates = candidate_similarities[:2224]

# End timing
lsh_end_time = time.time()
lsh_total_time = lsh_end_time - lsh_start_time

print(f"Total Time Taken: {lsh_total_time} seconds")

Total Time Taken: 1.3570959568023682 seconds


### 8. 

Compare the precision and the running time in Parts 1 and 2.

## Task 3. Data preperation

For this task, use the Pima Indians Diabetes Database.

### 1. 

Compute the correlation between the different columns after removing the `outcome` column.

### 2.

Remove the disguised values from the table. We need to remove the values that equal to `0` from columns `BloodPressure`, `SkinThickness` and `BMI` as these are missing values but they have been replaced by the value `0`. Remove the value but keep the record (i.e.) change the value to `null`.

### 3.

Fill the cells with `null` using the mean values of the records that have the same class label.

### 4.

Compute the correlation between the different columns.

### 5.

Compare the values from this step with the values in the first step (just mention the most important changes (if any)) and comment on your findings.